---

# 🛠️ Custom Serialization

### 🎯 Intent

Control **how models and fields render** to dict/JSON — mask secrets, format values, reshape payloads, and optimize API contracts.

---

### 🧩 Core Components

1. **🧪 `@field_serializer`**

   * Customize output of a **single field**.
   * Use for masking (e.g., passwords), formatting decimals, money, phone numbers.

2. **🏗️ `@model_serializer`**

   * Override the **entire model’s output**.
   * Useful for reshaping payloads (e.g., internal vs public API).

3. **🔁 Serializer Modes**

   * `plain` → transform the raw value directly.
   * `wrap` → get `(value, serializer)` so you can tweak after default serialization.

4. **📦 Nested & Collections**

   * Serializers apply recursively in lists/dicts/sets of models.
   * Combine with `include`/`exclude` for fine-grained pruning.

5. **🧭 Aliases + Serializers**

   * Works seamlessly with `by_alias=True` and field aliases.
   * Can remap fields to external names in `@model_serializer`.

6. **🧰 Works with Dump Options**

   * Serializers still honor:

     * `exclude_none`
     * `exclude_unset`
     * `exclude_defaults`
     * `mode="python" | "json"`

7. **🛡️ Redaction & PII**

   * Central place to mask sensitive data (tokens, emails, phones).
   * Raw values remain safe internally; only masked when serialized.

8. **⏱ Formatting Common Types**

   * Datetime → ISO strings or custom format.
   * Decimal/Money → fixed precision.
   * Enum → return `value` or a friendly label.

9. **⚙️ Tips**

   * Use **field serializers** for local tweaks.
   * Use **model serializer** for reshaping.
   * Prefer `wrap` mode to extend default behavior.

---